In [1]:
import sys
sys.path.append('..')

In [2]:
from tofina.components import asset, preference
from tofina.theory import varianceMinimization
import tofina.utils as utils
from tofina.macros import portfolioOptimization, optionPricing
import pandas as pd
import numpy as np
import torch

minVarianceFilePath1 = "minVariancePortfolio1.csv"
covariance1 = torch.tensor([[5, 1], [1, 10]]).float() / 100
mean=0.03


portfolio_ = varianceMinimization.GenerateVarianceMinimizationPortfolio(
    mean, covariance1, torch.tensor([1.0, 1.0]), torch.tensor([1.0, 1.0])
)
optim = portfolioOptimization.optimizeStockPortfolioRiskAverse(
    portfolio_, minVarianceFilePath1
)



../tofina/utils.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  params[key] = torch.nn.Parameter(torch.tensor(val), requires_grad=False)
  9%|▉         | 88/1000 [00:00<00:03, 267.86it/s]


In [3]:
optim.optimizationResult

{'initial_Stock_Company0weight': 0.5,
 'initial_Stock_Company1weight': 0.5,
 'initial_loss': -0.019090916961431503,
 'final_Stock_Company0weight': 0.6950502991676331,
 'final_Stock_Company1weight': 0.30494970083236694,
 'final_loss': -0.020339922979474068,
 'converged': True}

In [1]:
import sys
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model
import datetime as dt
import torch
sys.path.append('..')

In [1]:
import sys
sys.path.append('..')
from tofina.components.backtest import Backtester
from tofina.extern.arch import forecastDecoratorGARCH
from tofina.extern.yfinance import loadHistoricalStockData
import datetime as dt
import numpy as np

SPY = loadHistoricalStockData("SPY")
start_date = dt.datetime(2023, 1, 1)
end_date = dt.datetime(2024, 1, 1)
split_date = start_date - dt.timedelta(days=1)
timestamps = SPY.index[
    np.logical_and(SPY.index >= start_date, SPY.index < end_date)
]
horizon = 20
simulations = 1000
arch_kwargs = dict(vol="Garch", p=3, o=0, q=3, dist="t", mean="AR", lags=5)
forecaster = forecastDecoratorGARCH(
    SPY, split_date, horizon, simulations, **arch_kwargs
)
backtester = Backtester(timestamps=timestamps)
backtester.stockDataFromDataFrame(SPY, "SPY")
backtester.registerForecaster(forecaster, ["SPY"])

[*********************100%%**********************]  1 of 1 completed


Iteration:      5,   Func. Count:     94,   Neg. LLF: 30356.489405585766
Iteration:     10,   Func. Count:    181,   Neg. LLF: 15244.176184581413
Iteration:     15,   Func. Count:    263,   Neg. LLF: 10144.559858087046
Iteration:     20,   Func. Count:    344,   Neg. LLF: 10075.081632452344
Iteration:     25,   Func. Count:    420,   Neg. LLF: 10073.148971941075
Optimization terminated successfully    (Exit mode 0)
            Current function value: 10073.14896934825
            Iterations: 27
            Function evaluations: 449
            Gradient evaluations: 27


/opt/anaconda3/lib/python3.8/site-packages/arch/__future__/_utility.py:11: FutureWarning: 
The default for reindex is True. After September 2021 this will change to
False. Set reindex to True or False to silence this message. Alternatively,
you can use the import comment

from arch.__future__ import reindexing

to globally set reindex to True and silence this warning.

  warnings.warn(


In [2]:
dir(backtester)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'backtest',
 'historicalTrajectory',
 'horizon',
 'pointInTimePortfolio',
 'registerForecaster',
 'stockDataFromDataFrame',
 'ticker_forecaster_map',
 'timestamps']

In [8]:
timestamps = list(backtester.historicalTrajectory["SPY"].keys())
backtester.historicalTrajectory["SPY"][timestamps[0]]

array([380.82000732, 383.76000977, 379.38000488, 388.07998657,
       387.85998535, 390.57998657, 395.51998901, 396.95999146,
       398.5       , 397.76998901, 391.48999023, 388.64001465,
       395.88000488, 400.63000488, 400.20001221, 400.3500061 ,
       404.75      , 405.67999268, 400.58999634, 406.48001099])

In [9]:
backtester.historicalTrajectory["SPY"][timestamps[1]]

array([383.76000977, 379.38000488, 388.07998657, 387.85998535,
       390.57998657, 395.51998901, 396.95999146, 398.5       ,
       397.76998901, 391.48999023, 388.64001465, 395.88000488,
       400.63000488, 400.20001221, 400.3500061 , 404.75      ,
       405.67999268, 400.58999634, 406.48001099, 410.79998779])

In [30]:
backtester.historicalTrajectory["SPY"][timestamps[-60]]

array([424.5       , 429.54000854, 432.29000854, 434.54000854,
       436.32000732, 433.66000366, 431.5       , 436.04000854,
       436.01998901, 430.20999146, 426.42999268, 421.19000244,
       420.45999146, 423.63000488, 417.54998779, 412.54998779,
       410.67999268, 415.58999634, 418.20001221, 422.66000366])

In [32]:
backtester.ticker_forecaster_map["SPY"](timestamps[-60], "SPY")[:,-1]

tensor([443.9474, 457.5859, 446.0673, 438.8613, 439.2987, 430.0964, 432.4268,
        438.6432, 435.9447, 440.2409, 430.6357, 426.3887, 443.7874, 426.7075,
        413.1866, 438.6957, 429.6552, 434.3187, 428.0150, 435.1862, 452.0602,
        413.4997, 431.3378, 434.6848, 417.2231, 519.0523, 424.5539, 441.1955,
        431.1753, 444.0426, 416.0171, 442.1600, 422.8217, 451.5781, 456.9208,
        431.8348, 409.3048, 428.8025, 429.1193, 454.1040, 432.7941, 440.6176,
        418.7314, 427.9550, 426.6230, 435.2889, 436.4704, 459.0629, 440.9856,
        423.2804, 424.9035, 458.4294, 436.0223, 346.9683, 436.9718, 442.7464,
        475.0686, 429.8578, 451.8465, 407.5223, 439.2906, 447.4432, 435.8425,
        441.9217, 449.6732, 431.2496, 422.1707, 430.2186, 403.1694, 441.4050,
        423.7525, 424.8548, 421.9578, 427.7810, 422.8362, 422.9586, 428.4357,
        434.3880, 423.0369, 419.6593, 426.1211, 438.0880, 436.1357, 455.5335,
        419.3156, 448.2975, 417.1870, 430.5622, 430.5795, 410.19